## ЦИФРОВОЙ ПРОРЫВ 2022
# **Чемпионат в республике Саха (Якутия)**
# Разработка алгоритма по прогнозированию карьерной траектории сотрудника
Технологии искусственного интеллекта трансформируют сферу управления персоналом, позволяя решать практические HR-задачи. Уже сейчас умные алгоритмы могут подобрать кадры, спланировать размер фонда заработной платы, выбрать образовательные курсы, оценить мотивацию, эффективность труда и даже спрогнозировать карьерное развитие специалиста.

Аналитика по определению карьерной траектории строится, в том числе, на основе данных по использованию различных корпоративных информационных систем (система мониторинга рабочего времени, сервисы видеоконференции связи, IP-телефонии, СЭД, СКУД, электронная почта). Таким образом, для некоторых профессий, связанных с активным использованием электронных устройств (программист, бухгалтер, маркетолог и т.д.), можно выделить цифровой профиль успешных специалистов, выполняющих задачи своевременно и качественно и получающих новые должности.

На основе совокупности информации о кадровых назначениях, активности и вовлеченности сотрудников в производственный процесс, участникам чемпионата нужно предложить алгоритм по прогнозу изменений должности работников.

# **Data analysis**
**Импортируем все необходимые библиотеки и модули**

In [33]:
import pandas as pd
import numpy as np
import time

from imblearn.over_sampling import SMOTE 
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

**Загрузка данных в датафреймы**

In [34]:
df_train = pd.read_csv("../input/yakutsk/train_dataset_train.csv")
df_test = pd.read_csv("../input/yakutsk/test_dataset_test.csv")
calls = pd.read_csv("../input/yakutsk/Calls.csv")
connect = pd.read_csv("../input/yakutsk/ConnectionTime.csv")
educat = pd.read_csv("../input/yakutsk/Education.csv")
skud = pd.read_csv("../input/yakutsk/SKUD.csv")
tasks = pd.read_csv("../input/yakutsk/Tasks.csv")
timen = pd.read_csv("../input/yakutsk/TimenNetwork.csv")
working = pd.read_csv("../input/yakutsk/WorkingDay.csv")

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3552: DtypeWarning: Columns (1,7) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


**Смотрим первые записи**

In [35]:
df_train.head(3)

,id,type
0,ОРГ1-02050,2
1,ОРГ1-02783,2
2,ОРГ2-06173,0


# **Подготовка данных**

**Записываем id в отдельный лист**

In [36]:
test_id = df_test[["id"]]

**Считаем среднее количество звонков в день и среднее количество времени за телефоном**

In [37]:
calls["CallTime"] = calls["CallTime"].apply(lambda x: float(x.replace(',', '.')))

In [38]:
calls["avg_calls"] = calls.groupby(["id"])["NumberOfCalls"].transform('mean')
calls["avg_calltime"] = calls.groupby(["id"])["CallTime"].transform('mean')

In [39]:
calls.head(3)

,Date,CallTime,NumberOfCalls,Вид учета времени,InOut,id,avg_calls,avg_calltime
0,"2021-08-16 00:00:00,000",0.000278,1,Будни,ToUser,ОРГ1-01945,4.486166,0.084439
1,"2021-09-21 00:00:00,000",0.000278,1,Будни,ToUser,ОРГ1-01945,4.486166,0.084439
2,"2021-01-11 00:00:00,000",0.000278,1,Будни,ToUser,ОРГ1-01945,4.486166,0.084439


# **Работа с df_connection**
Параметр *late_count* будет хранить общее количество времени опоздания

In [40]:
connect["Время опоздания"] = connect["Время опоздания"].apply(lambda x: float(str(x).replace(',', '.')))
connect["Время опоздания"] = connect["Время опоздания"].apply(lambda x: float(str(x).replace('nan', '0')))

In [41]:
connect["late_count"] = connect.groupby(["id"])["Время опоздания"].transform('sum')

In [42]:
connect.head(3)

,dateNum,maxLogOff,Нормативное время начала раб.дня,Фактич. время начала раб.дня,Время опоздания,Признак опоздания,Вых/Будни,id,late_count
0,"2021-12-15 00:00:00,000","2021-12-15 17:30:27,246","1899-12-30 08:30:00,000","1899-12-30 08:24:18,606",0.0,NaN,Будни,ОРГ1-02782,0.393761
1,"2021-12-15 00:00:00,000","2021-12-15 16:30:13,330","1899-12-30 08:30:00,000","1899-12-30 01:05:20,513",0.0,NaN,Будни,ОРГ1-01407,364.750232
2,"2021-12-15 00:00:00,000","2021-12-15 15:27:37,246","1899-12-30 08:30:00,000","1899-12-30 05:47:33,156",0.0,NaN,Будни,ОРГ1-01909,366.858304


# **Работа с df_tasks**
Категоризируем параметр "статус по просрочке", чтобы 0 обозначал, что задание выполнено в срок, а 1 - просроченное. Параметр *overdue_count* будет обозначать количество просроченнных заданий.

In [43]:
tasks["Статус по просрочке"] = tasks["Статус по просрочке"].astype("category").cat.codes
tasks["overdue_count"] = tasks.groupby(["id"])["Статус по просрочке"].transform('sum')

In [44]:
tasks.head(3)

,Статус по просрочке,Срок плановый,"Просрочено, дней",ДлительностьПросрочки,ID задачи,Вид документа,Дата старта задания,Дата завершения задания плановая,Дата завершения задания фактическая,Состояние задания,id,overdue_count
0,0,NaN,0,без нарушения срока,E1DE844D-EE2D-4C41-AEDF-93F246749F0E,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588,0.0
1,0,NaN,0,без нарушения срока,7A92343C-8C9A-46E7-AC81-8F50F95009D0,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588,0.0
2,0,NaN,0,без нарушения срока,5CE64E52-D2D1-4DCC-B2C8-34734AA39AC0,Служебная записка,"2021-12-10 00:00:00,000",NaN,"2021-12-10 00:00:00,000",Завершено,ОРГ1-02588,0.0


# **Работа с df_working**
Создаём новый параметр *monitorTime_count* - общее время активности

In [45]:
working["monitorTime_count"] = working.groupby(["id"])["monitorTime"].transform('sum')

In [46]:
working.head(3)

,startTime,activeTime,Вых/Будни,monitorTime,id,monitorTime_count
0,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-01553,3802145
1,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-02112,3791253
2,"2021-11-30 00:00:00,000",2,Будни,2,ОРГ1-02112,3791253


# **Работа с df_timen**
Создаём новый параметр *monitorTime_avg* - среднее время активности

In [47]:
timen["monitorTime_avg"] = timen.groupby(["id"])["monitor_Time"].transform('mean')

In [48]:
timen.head(3)

,Вых/Будни,monitor_Time,startTime,id,monitorTime_avg
0,Будни,300,"2021-08-16 00:00:00,000",ОРГ1-01402,5566.280639
1,Будни,300,"2021-08-18 00:00:00,000",ОРГ1-01402,5566.280639
2,Будни,300,"2021-08-19 00:00:00,000",ОРГ1-01402,5566.280639


# **Работа с df_skud**
Категоризируем столбец "Вых/Будни" и считаем количество проработанных дней в новый параметр *days_count*

In [49]:
skud["Вых/Будни"] = skud["Вых/Будни"].astype("category").cat.codes
skud["days_count"] = skud.groupby(["id"])["Вых/Будни"].transform('count')

In [50]:
skud.head(3)

,Дата,Приход.1,Уход.1,Длительность общая,Длительность раб.дня без обеда,Вых/Будни,id,days_count
0,"2021-03-01 00:00:00,000","1899-12-30 08:11:00,000","1899-12-30 17:32:00,000","9,35","8,35",0,ОРГ1-00791,23
1,"2021-03-01 00:00:00,000","1899-12-30 08:25:00,000","1899-12-30 17:32:00,000","9,1166666666666671","8,1166666666666671",0,ОРГ1-01826,9
2,"2021-03-01 00:00:00,000","1899-12-30 08:31:00,000","1899-12-30 17:32:00,000","9,0166666666666657","8,0166666666666657",0,ОРГ1-00713,32


# **Работа с df_education**
Пронумерумем каждый вид образования и создадим несколько новых параметров:
* retraining - заменяем на ноль, если образование не является переподготовкой, иначе оставляем присвоенный номер
* Вид образования - меняем на ноль, если образование это переподготовка, иначе оставляем присвоенный номер
* educat_count - Количество образований
* chief - 1, если человек является руководителем
* kind_educa - "лучшее" образование
* retraining_count - количество переподготовок
* number_chief - категоризация руководителей

In [51]:
educat["Вид образования"] = educat["Вид образования"].fillna(-1)
educat["Вид образования"] = educat["Вид образования"].map({"Высшее образование - бакалавриат": 5,
                                                                      "Среднее профессиональное образование": 4,
                                                                      "Высшее образование": 5,
                                                                      "Повышение квалификации": 8,
                                                                      "Переподготовка": 7, "Послевузовское образование": 7,
                                                                      "Дополнительное профессиональное образование": 7,
                                                                      "Высшее образование - специалитет, магистратура": 5,
                                                                      "Среднее общее образование": 2,
                                                                      "Начальное профессиональное образование": 2,
                                                                      "Аспирантура": 6,
                                                                      "Среднее (полное) общее образование": 2,
                                                                      "Неполное высшее образование": 3,
                                                                      "Основное общее образование": 2,
                                                                      "Профессиональное обучение": 3,
                                                                      "Начальное общее образование": 1})
educat = educat.drop_duplicates()
educat["retraining"] = educat["Вид образования"].apply(lambda x: x if x > 6 else 0)
educat["Вид образования"] = educat["Вид образования"].apply(lambda x: x if x <= 6 else 0)
educat["educat_count"] = educat["id"].map(educat[educat["Вид образования"] != 0].groupby(["id"])["Вид образования"].count())

In [52]:
educat["chief"] = educat["Табельный номер руководителя"] == educat["id"]
educat["chief"] = educat["chief"].astype("int")
educat["kind_educat"] = educat.groupby("id")["Вид образования"].transform("max")
educat["retraining_count"] = educat.groupby("id")["retraining"].transform("count")
educat["Табельный номер руководителя"].fillna(-1)
educat["number_chief"] = educat["Табельный номер руководителя"].astype("category").cat.codes
educat = educat[["id", "retraining_count", "educat_count", "chief", "number_chief"]]
educat = educat.drop_duplicates()

In [53]:
educat.head(3)

,id,retraining_count,educat_count,chief,number_chief
0,ОРГ1-00131,1,1.0,0,-1
1,ОРГ1-03220,1,1.0,0,-1
2,ОРГ1-03008,1,1.0,0,-1


# **Подготовка датасетов для тренировки**

**Сливаем нужные столбцы в один датасет**

In [54]:
train1 = df_train.merge(calls[["id", "avg_calls","avg_calltime"]], how="left", on="id").drop_duplicates().reset_index(drop=True)
test1 = test_id.merge(calls[["id", "avg_calls","avg_calltime"]], how="left", on="id").drop_duplicates().reset_index(drop=True)

In [55]:
train2 = train1.merge(connect[["id", "late_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)
test2 = test1.merge(connect[["id", "late_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)

In [56]:
train3 = train2.merge(tasks[["id", "overdue_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)
test3 = test2.merge(tasks[["id", "overdue_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)

In [57]:
train4 = train3.merge(working[["id", "monitorTime_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)
test4 = test3.merge(working[["id", "monitorTime_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)

In [58]:
train5 = train4.merge(timen[["id", "monitorTime_avg"]], how="left", on="id").drop_duplicates().reset_index(drop=True)
test5 = test4.merge(timen[["id", "monitorTime_avg"]], how="left", on="id").drop_duplicates().reset_index(drop=True)

In [59]:
train6 = train5.merge(skud[["id", "days_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)
test6 = test5.merge(skud[["id", "days_count"]], how="left", on="id").drop_duplicates().reset_index(drop=True)

In [60]:
train7 = train6.merge(educat, how="left", on="id").drop_duplicates().reset_index(drop=True)
test7 = test6.merge(educat, how="left", on="id").drop_duplicates().reset_index(drop=True)

In [61]:
trainFinal = train7.fillna(0)
testFinal = test7.fillna(0)
trainFinal.head(3)

,id,type,avg_calls,avg_calltime,late_count,overdue_count,monitorTime_count,monitorTime_avg,days_count,retraining_count,educat_count,chief,number_chief
0,ОРГ1-02050,2,3.640000,0.178522,17.369871,0.0,12244189.0,24445.805310,31.0,2.0,2.0,0.0,-1.0
1,ОРГ1-02783,2,4.605327,0.120907,42.616769,50.0,11458725.0,28655.859375,0.0,1.0,1.0,0.0,97.0
2,ОРГ2-06173,0,0.000000,0.000000,91.630440,0.0,0.0,0.000000,0.0,1.0,1.0,0.0,136.0


In [62]:
testFinal.head(3)

,id,avg_calls,avg_calltime,late_count,overdue_count,monitorTime_count,monitorTime_avg,days_count,retraining_count,educat_count,chief,number_chief
0,ОРГ1-02649,1.333333,0.036759,0.000000,0.0,7259027.0,21335.122807,32.0,0.0,0.0,0.0,0.0
1,ОРГ2-05929,0.000000,0.000000,94.338109,0.0,2410531.0,14605.670000,0.0,1.0,1.0,0.0,134.0
2,ОРГ2-05859,0.000000,0.000000,170.895793,19.0,4281484.0,15538.574899,0.0,1.0,1.0,0.0,144.0


In [63]:
X = trainFinal.drop(["id", "type"], axis=1).values
Y = trainFinal["type"].values

In [64]:
X_test = testFinal.drop(["id"], axis=1).values

# **Over-sampling (устранение дисбаланса классов)**
Для устранение дисбаланса классов воспльзуемся SMOTE из библиотеки imblearn, этот алгоритм создает дополнительные синтетические наблюдения меньших классов.

In [65]:
oversample = SMOTE()
X, Y = oversample.fit_resample(X, Y)

# **Machine Learning**
В качестве модели были использованы алгоритмы RandomForectClassifier, Support Vector Classification и Logistic Regression. Затем прибегли к использованию ансамбля моделей, когда результаты сразу нескольких из них участвуют в формировании конечного результата.

In [66]:
ss = StandardScaler()
X_scaled = ss.fit_transform(X)
X_t_scaled = ss.transform(X_test)

In [67]:
clf1 = RandomForestClassifier(n_estimators=100)
clf2 = make_pipeline(ss, SVC(gamma='auto', probability=True))
clf3 = LogisticRegression(random_state=0)
eclf = VotingClassifier(estimators=[('rfc', clf1), ('svc', clf2), ('logreg', clf3)], voting='soft')
eclf = eclf.fit(X_scaled, Y)
Y_pred = eclf.predict(X_t_scaled)

# **Формировка результатов**

In [68]:
sub = pd.DataFrame({'id':test_id["id"], 'type':Y_pred})
sub.to_csv('VotingClassifier (rfc, svc, logreg) v2.csv', index = False)